# Physics Calibration Notebook

Estimate realistic motion caps (speed/acceleration) by sampling tracking frames and summarizing by position.

## Setup

In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

REPO_ROOT = Path('..').resolve().parents[0]
TRACKING_DIR = REPO_ROOT / 'analytics' / 'data' / '114239_nfl_competition_files_published_analytics_final' / 'train'
print('Tracking dir:', TRACKING_DIR)

## Load Sample Frames
Limit to a couple of weekly files to keep iteration snappy.

In [ ]:
files = sorted(TRACKING_DIR.glob('input_2023_w*.csv'))[:2]
cols = ['game_id','play_id','nfl_id','frame_id','player_position','s','a']
dtypes = {
    'game_id':'int64','play_id':'int64','nfl_id':'int64','frame_id':'int16',
    'player_position':'category','s':'float32','a':'float32'
}
frames = []
for path in files:
    df = pd.read_csv(path, usecols=cols, dtype=dtypes)
    frames.append(df)
tracking = pd.concat(frames, ignore_index=True)
tracking['speed'] = tracking['s'].clip(lower=0)
tracking['accel'] = tracking['a']
tracking.head()

## Position-Level Caps

In [ ]:
quantiles = tracking.groupby('player_position').agg({
    'speed': ['mean','median',lambda x: x.quantile(0.95), lambda x: x.max()],
    'accel': ['mean','median',lambda x: x.quantile(0.95), lambda x: x.max()],
})
quantiles.columns = ['speed_mean','speed_median','speed_p95','speed_max','accel_mean','accel_median','accel_p95','accel_max']
quantiles.sort_values('speed_p95', ascending=False).head(10)

## Visualization
Inspect distributions for a couple of position groups.

In [ ]:
sel_positions = ['CB','S','LB','WR']
subset = tracking[tracking['player_position'].isin(sel_positions)].sample(n=20000, random_state=42)
plt.figure(figsize=(10,5))
sns.kdeplot(data=subset, x='speed', hue='player_position', common_norm=False)
plt.title('Speed Distributions by Position (sample)')
plt.show()

plt.figure(figsize=(10,5))
sns.kdeplot(data=subset, x='accel', hue='player_position', common_norm=False)
plt.title('Acceleration Distributions by Position (sample)')
plt.show()